In [24]:
import polars as pl
import os
from tqdm import tqdm
import chardet
import re
from dataclasses import dataclass
from typing import List, Optional

In [ ]:
texts_dir = os.path.expanduser("~/Downloads/raw_texts")
all_text_files = [os.path.join(texts_dir, f) for f in os.listdir(texts_dir) if f.endswith(".txt")]

script_items = {
    "filename": [],
    "contents": []
}

for filename in tqdm(all_text_files):
    with open(filename, "rb") as f:
        raw_data = f.read()
        encoding = chardet.detect(raw_data)["encoding"]
    with open(filename, "r", encoding=encoding) as f:
        script_items["filename"].append(filename)
        script_items["contents"].append(f.read())

In [128]:

@dataclass
class DialogueLine:
    character: str
    content: str
    parenthetical: Optional[str] = None
    line_number: int = 0

@dataclass
class SceneElement:
    type: str  # Can be "scene_heading", "action", "dialogue", "transition"
    content: str
    line_number: int
    dialogue_data: Optional[DialogueLine] = None  # Used only for dialogue elements

@dataclass
class Scene:
    heading: SceneElement
    elements: List[SceneElement]

class ScreenplayParser:
    def __init__(self, text: str):
        self.lines = text.split('\n')
        self.current_line = 0
        self.total_lines = len(self.lines)
        self.scenes: List[Scene] = []
        
    def parse(self) -> List[Scene]:
        """Parse the entire screenplay and return a list of scenes."""
        # Skip any leading blank lines or title page content
        while self.current_line < self.total_lines:
            if self._is_scene_heading(self.current_line):
                self._parse_scene()
            self.current_line += 1
        return self.scenes
    
    def _is_scene_heading(self, line_num: int) -> bool:
        """Check if the current line is a scene heading."""
        line = self.lines[line_num].strip()
        return bool(re.match(r'^(INT\.|EXT\.).+', line))
    
    def _is_character_name(self, line_num: int) -> bool:
        """Check if the current line is a character name."""
        line = self.lines[line_num].strip()
        return bool(line and line.isupper() and not line.startswith('(') 
                   and not self._is_scene_heading(line_num)
                   and not self._is_transition(line_num))
    
    def _is_parenthetical(self, line_num: int) -> bool:
        """Check if the current line is a parenthetical."""
        line = self.lines[line_num].strip()
        return line.startswith('(') and line.endswith(')')
    
    def _is_transition(self, line_num: int) -> bool:
        """Check if the current line is a transition."""
        line = self.lines[line_num].strip()
        return bool(re.match(r'^(FADE|DISSOLVE|CUT).+', line))
    
    def _merge_consecutive_elements(self, elements: List[SceneElement]) -> List[SceneElement]:
        """Merge consecutive elements of the same type."""
        if not elements:
            return elements
            
        merged = []
        current = elements[0]
        current_lines = [current.content]
        current_start_line = current.line_number
        
        for next_elem in elements[1:]:
            if (next_elem.type == current.type and 
                next_elem.line_number == current_start_line + len(current_lines)):
                current_lines.append(next_elem.content)
            else:
                merged.append(SceneElement(
                    type=current.type,
                    content=" ".join(current_lines),
                    line_number=current_start_line,
                    dialogue_data=current.dialogue_data
                ))
                current = next_elem
                current_lines = [current.content]
                current_start_line = current.line_number
                
        # Don't forget to add the last group
        merged.append(SceneElement(
            type=current.type,
            content=" ".join(current_lines),
            line_number=current_start_line,
            dialogue_data=current.dialogue_data
        ))
        
        return merged
    
    def _parse_dialogue_block(self) -> SceneElement:
        """Parse a complete dialogue block including character name, parentheticals, and dialogue."""
        character_line = self.current_line
        character = self.lines[self.current_line].strip()
        self.current_line += 1
        
        parenthetical = None
        dialogue_lines = []
        
        while self.current_line < self.total_lines:
            line = self.lines[self.current_line].strip()
            
            if not line:
                break
                
            if self._is_parenthetical(self.current_line):
                parenthetical = line
            else:
                dialogue_lines.append(line)
                
            self.current_line += 1
            
            # Check next line - if blank or new character/scene heading, end dialogue block
            if self.current_line >= self.total_lines:
                break
            next_line = self.lines[self.current_line].strip()
            if (not next_line or self._is_character_name(self.current_line) 
                or self._is_scene_heading(self.current_line)):
                break
        
        dialogue_content = " ".join(dialogue_lines)
        
        return SceneElement(
            type="dialogue",
            content=dialogue_content,
            line_number=character_line,
            dialogue_data=DialogueLine(
                character=character,
                content=dialogue_content,
                parenthetical=parenthetical,
                line_number=character_line
            )
        )
    
    def _parse_scene(self):
        """Parse a single scene."""
        # Parse scene heading
        heading = SceneElement(
            type="scene_heading",
            content=self.lines[self.current_line].strip(),
            line_number=self.current_line
        )
        
        elements = []
        self.current_line += 1
        
        # Parse scene content until we hit the next scene heading or end of script
        while self.current_line < self.total_lines:
            if self._is_scene_heading(self.current_line):
                break
                
            line = self.lines[self.current_line].strip()
            
            # Skip blank lines
            if not line:
                self.current_line += 1
                continue
                
            if self._is_character_name(self.current_line):
                dialogue_element = self._parse_dialogue_block()
                elements.append(dialogue_element)
            elif self._is_transition(self.current_line):
                elements.append(SceneElement(
                    type="transition",
                    content=line,
                    line_number=self.current_line
                ))
                self.current_line += 1
            else:
                # Assume it's action description
                elements.append(SceneElement(
                    type="action",
                    content=line,
                    line_number=self.current_line
                ))
                self.current_line += 1
                
        # Merge consecutive elements before adding to scene
        elements = self._merge_consecutive_elements(elements)
        self.scenes.append(Scene(heading=heading, elements=elements))

In [74]:
with open("script_sample.txt", "w") as f:
    f.write(text_sample[:20000])

In [ ]:
scripts_pl = pl.DataFrame(script_items)
text_sample = scripts_pl["contents"][0]
parser = ScreenplayParser(text_sample)


scenes = parser.parse()
for scene in scenes:
    for element in scene.elements:
        if element.dialogue_data:
            print(element.dialogue_data)
            print(f"{element.type}: {element.dialogue_data.character} - {element.dialogue_data.content}")
        else:
            print(f"{element.type}: {element.content}")


In [ ]:
from typing import Sequence, TypedDict


class Message(TypedDict):
    role: str
    content: str


def conversation_format_script(script: List[Scene]) -> Sequence[Message]:
    conversation: Sequence[Message] = []
    for scene in script:
        for element in scene.elements:
            if element.dialogue_data:
                role = element.dialogue_data.character
                content = element.dialogue_data.content
                if element.dialogue_data.parenthetical:
                    content += f" ({element.dialogue_data.parenthetical})"
                conversation.append({"role": role, "content": content})
            else:
                role = f"<{element.type}>"
                conversation.append({"role": role, "content": element.content})
    return conversation


conversation_format_script(scenes)

In [ ]:
out = []
for name, screenplay in tqdm(scripts_pl.rows()):
    parser = ScreenplayParser(screenplay)
    scenes = parser.parse()
    conversation = conversation_format_script(scenes)
    file_name = os.path.basename(name)
    if len(conversation) > 0:
        out.append({"filename": file_name, "conversation": conversation})

out_pl = pl.DataFrame(out)
out_pl.write_parquet("screenplay_conversations.parquet")

In [ ]:
out_pl